In [6]:

from utils import get_device
from model_task_3 import train_model
from data_processing import get_loaders, class_cols
from model_task_3 import create_model, train_model
import torch

import pandas as pd
from tqdm import tqdm

In [7]:

device = get_device()

train_loader, val_loader, train_df, val_df = get_loaders(
    image_size=(384, 384),
    num_workers=0,
)
class_cols_order = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"]
num_classes = len(class_cols_order)
model = create_model(num_classes=num_classes, model_name="vit_base_patch16_384")


class_counts = [int(train_df[c].sum()) for c in class_cols_order]

#base_model = create_model(num_classes=num_classes, model_name="vit_base_patch16_384")
#ckpt = torch.load("best_model.pth", map_location=device)
#base_model.load_state_dict(ckpt["model_state"])


model, history2 = train_model(
    train_loader,
    val_loader,
    num_classes=num_classes,
    device=device,
    epochs=15,                 
    class_counts=class_counts,
    #model=base_model,          
    save_path="best_model.pth" 
)

Epoch 1/30 train_loss=0.2618 val_loss=2.9973 train_acc=0.0497 val_acc=0.0592
  ✓ New best val_acc=0.0592, model saved to best_model.pth


Epoch 2/30 train_loss=0.1565 val_loss=3.4493 train_acc=0.0707 val_acc=0.0206


Epoch 3/30 train_loss=0.1337 val_loss=2.5317 train_acc=0.0996 val_acc=0.1184
  ✓ New best val_acc=0.1184, model saved to best_model.pth


Epoch 4/30 train_loss=0.1256 val_loss=2.7537 train_acc=0.1017 val_acc=0.0732


Epoch 5/30 train_loss=0.1192 val_loss=2.4248 train_acc=0.1074 val_acc=0.1337
  ✓ New best val_acc=0.1337, model saved to best_model.pth


...

Epoch 15/30 train_loss=0.0815 val_loss=2.0767 train_acc=0.2162 val_acc=0.2974


Epoch 16/30 train_loss=0.0752 val_loss=2.2914 train_acc=0.2676 val_acc=0.2735


Epoch 17/30 train_loss=0.0696 val_loss=1.9433 train_acc=0.2788 val_acc=0.2954


Epoch 18/30 train_loss=0.0610 val_loss=2.1399 train_acc=0.2750 val_acc=0.2861


KeyboardInterrupt: 

In [5]:
# Evaluate on test set (if available)
# This cell will request loaders including the test set and compute test loss/accuracy
from data_processing import TASK_3_TEST_LABELS_DIR

# Try to get loaders and request the test CSV explicitly (module-level constant used as default)
loaders = get_loaders(
    image_size=(384, 384),
    num_workers=0,
    test_csv_path=TASK_3_TEST_LABELS_DIR,
)

# unpack safely depending on what get_loaders returned
test_loader = None
if isinstance(loaders, tuple) and len(loaders) == 6:
    train_loader, val_loader, test_loader, train_df, val_df, test_df = loaders
elif isinstance(loaders, tuple) and len(loaders) == 4:
    train_loader, val_loader, train_df, val_df = loaders
    test_loader = None
else:
    # unexpected shape
    test_loader = None

if test_loader is None:
    print("No test loader available. Provide a test CSV/images or adjust get_loaders to return a test loader.")
else:
    import torch
    import torch.nn as nn

    device = get_device()
    model.to(device)
    model.eval()

    criterion = nn.CrossEntropyLoss()
    total = 0
    correct = 0
    running_loss = 0.0

    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            running_loss += loss.item() * xb.size(0)
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += xb.size(0)

    test_loss = running_loss / total if total > 0 else 0.0
    test_acc = correct / total if total > 0 else 0.0
    print(f"Test samples: {total}")
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test accuracy: {test_acc:.4%}")


FileNotFoundError: [Errno 2] No such file or directory: "'data/Task_3/ISIC2018_Task3_Training_GroundTruth.csv' # Gian"

In [ ]:
for views, labels in train_loader:
    # views is a list/tuple of tensors: [weak_batch, strong_batch]
    weak_imgs, strong_imgs = views[0].to(device), views[1].to(device)
    labels = labels.to(device)